# **Actividad 4 | Métricas de calidad de resultados**

José Ricardo Munguía Marín
A01795660@tec.mx


## 1 Construcción de la muestra M

Construir una muestra M que sea representativa de la población P (a partir del dataset que recolectaste desde el inicio del curso).

Esta parte viene de las actividades anteriores.

In [1]:
# IMPORTACIONES
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, year, when, udf
from pyspark.sql.types import StringType
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler, MinMaxScaler
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator

In [2]:

# ----------------------------------------
# SESIÓN SPARK
# ----------------------------------------
spark = SparkSession.builder.appName("MLlibEquipo52").getOrCreate()

# ----------------------------------------
# CARGA Y UNIÓN DE DATOS
# ----------------------------------------
books = spark.read.csv("books_data.csv", header=True, inferSchema=True)
ratings = spark.read.csv("Books_rating.csv", header=True, inferSchema=True)

books = books.withColumnRenamed("title", "Title")
df = ratings.join(books.select("Title", "categories", "publishedDate", "ratingsCount"), on="Title", how="inner")

# FILTRADO DE NULOS PARA VARIABLES CLAVE
# ----------------------------------------
df_clean = df.filter(col("ratingsCount").isNotNull() & col("review/score").isNotNull())

# PARTICIONAMIENTO SEGÚN CRITERIOS DE LA ACTIVIDAD 2
# ----------------------------------------
df_clean = df_clean.withColumn("ratings_partition", when(col("ratingsCount") <= 1000, "baja").otherwise("alta"))
df_clean = df_clean.withColumn("score_partition", when(col("review/score") < 4.0, "baja").otherwise("alta"))

part_A = df_clean.filter((col("ratings_partition") == "baja") & (col("score_partition") == "baja"))
part_B = df_clean.filter((col("ratings_partition") == "baja") & (col("score_partition") == "alta"))
part_C = df_clean.filter((col("ratings_partition") == "alta") & (col("score_partition") == "baja"))
part_D = df_clean.filter((col("ratings_partition") == "alta") & (col("score_partition") == "alta"))

# MUESTREO POR PARTICIÓN
# ----------------------------------------
sample_A = part_A.sample(False, 0.3, seed=42) #Partición 1: muestreo Aleatorio Simple al 30%
sample_B = part_B.sample(False, 0.05, seed=42) #Partición 2: muestreo Aleatorio Simple al 5%

#Partición 3: muestreo sistemático, toma 1 de cada 5 registros mediante indexado con zipWithIndex() y filtro
sample_C = part_C.rdd.zipWithIndex().filter(lambda x: x[1] % 5 == 0).map(lambda x: x[0]).toDF()

#Partición 4: Muestra Completa
sample_D = part_D

muestra_M = sample_A.union(sample_B).union(sample_C).union(sample_D)


In [3]:

# ----------------------------------------
# TRANSFORMACIONES PREVIAS
# ----------------------------------------
muestra_M = muestra_M.withColumn("year", year(col("publishedDate")))

# LIMPIEZA Y CONVERSIÓN DE CATEGORIES A STRINGS SIMPLES
to_string_udf = udf(lambda x: str(x).strip("[]'").split(",")[0] if x else None, StringType())
muestra_M = muestra_M.withColumn("categories", to_string_udf("categories"))
muestra_M = muestra_M.filter(col("categories").isNotNull())

# CONVERSIÓN DE ratingsCount A TIPO NUMÉRICO
muestra_M = muestra_M.withColumn("ratingsCount", col("ratingsCount").cast("double"))

# BINARIZACIÓN DE LABEL
muestra_M = muestra_M.withColumn("label", when(col("review/score") >= 4.0, 1).otherwise(0))

# ELIMINAR NULOS EN COLUMNAS EXISTENTES
muestra_M = muestra_M.dropna(subset=["ratingsCount", "year", "categories"])

# PIPELINE DE TRANSFORMACIÓN
# ----------------------------------------
indexer = StringIndexer(inputCol="categories", outputCol="categories_index", handleInvalid="keep")
encoder = OneHotEncoder(inputCol="categories_index", outputCol="categories_encoded", handleInvalid="keep")
assembler = VectorAssembler(inputCols=["ratingsCount", "year", "categories_encoded"], outputCol="features_raw")
scaler = MinMaxScaler(inputCol="features_raw", outputCol="features")

pipeline = Pipeline(stages=[indexer, encoder, assembler, scaler])
model_prep = pipeline.fit(muestra_M)
muestra_final = model_prep.transform(muestra_M)


##2 Preparación del conjunto de entrenamiento y prueba


En esta actividad **no se volvió a dividir la muestra M por subconjuntos (A, B, C, D)**, ya que `M` fue construida de forma controlada, balanceada y representativa desde el inicio, aplicando diferentes técnicas de muestreo (simple, sistemático, censal, etc.) sobre grupos disjuntos.

Intentar particionar de nuevo por grupo generaba **duplicación de datos, sobrecarga de memoria y retrasos**, porque requería hacer joins con `muestra_final`, forzando un shuffle de datos innecesario. Además, no aportaba valor estadístico adicional.

Por ello, se optó por una división simple con `randomSplit([0.8, 0.2])` directamente sobre `muestra_final`. Esto mantiene la representatividad de los datos y mejora la eficiencia del pipeline sin afectar la validez del entrenamiento ni de la evaluación.

*Nota: Se intentó varías veces crear está Muestra, pero rompía el pipeline y generaba duplicación, se dejo corriendo por casi 24 Hrs sin resultados.*

In [4]:

# ----------------------------------------
# DIVISIÓN TRAIN/TEST
train_data, test_data = muestra_final.randomSplit([0.8, 0.2], seed=42)

# CONVERSIÓN DE LABEL A DOUBLE
train_data = train_data.withColumn("label", col("label").cast("double"))
test_data = test_data.withColumn("label", col("label").cast("double"))


##3 Selección de métricas para medir calidad de resultados

**Para el modelo supervisado:**

Se utilizaron las métricas de `accuracy`, `F1-score`, `precision ponderada` y `recall ponderado`.

- **Accuracy** indica el porcentaje de clasificaciones correctas.
- **F1-score** equilibra precisión y exhaustividad, útil si las clases están desbalanceadas.
- **Weighted Precision** mide qué tan confiables son las predicciones positivas, ponderadas por el soporte de cada clase.
- **Weighted Recall** refleja cuántos verdaderos positivos se capturaron, también ponderado.

En contextos de Big Data, usar métricas agregadas ponderadas ayuda a evaluar modelos sin procesar todos los datos de forma individual y mantiene una visión general balanceada del rendimiento.

**Para el modelo no supervisado (KMeans):**

Se eligió la métrica **Silhouette score**, ya que permite evaluar qué tan bien separados y compactos están los clústeres generados. Esta métrica va de -1 a 1, donde valores cercanos a 1 indican una buena separación entre grupos.  


## 4 Entrenamiento de Modelos de Aprendizaje

In [ ]:
# ----------------------------------------
# MODELO SUPERVISADO
# ----------------------------------------
dt = DecisionTreeClassifier(labelCol="label", featuresCol="features")
dt_model = dt.fit(train_data)
predictions = dt_model.transform(test_data)



In [8]:
# ----------------------------------------
# Métricas de Rendimiento para nuestro random forest
# ----------------------------------------

evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction")

for metric in ["accuracy", "f1", "weightedPrecision", "weightedRecall"]:
    evaluator.setMetricName(metric)
    print(f"{metric}: {evaluator.evaluate(predictions):.4f}")

accuracy: 0.7709
f1: 0.7759
weightedPrecision: 0.8133
weightedRecall: 0.7709


In [12]:

# MODELO NO SUPERVISADO
# ----------------------------------------
kmeans = KMeans(featuresCol="features", k=15, seed=42)
kmeans_model = kmeans.fit(muestra_final)
clusters = kmeans_model.transform(muestra_final)

evaluator_cluster = ClusteringEvaluator(featuresCol="features", metricName="silhouette", distanceMeasure="squaredEuclidean")
print("Silhouette score:", evaluator_cluster.evaluate(clusters))
clusters.groupBy("prediction").count().show()


Silhouette score: 0.7205232669620012
+----------+-----+
|prediction|count|
+----------+-----+
|        12| 2580|
|         1|56485|
|        13|35790|
|         6| 4402|
|         3| 8595|
|         5| 2029|
|         4| 1417|
|         8| 5212|
|        10| 4766|
|        11| 2255|
|        14| 1197|
|         2|36734|
|         9|10860|
|         7| 2033|
|         0|52769|
+----------+-----+



## **5 Análisis de resultados**

El modelo supervisado (árbol de decisión) mostró un buen desempeño general, con una **accuracy superior al 77%** y un **F1-score cercano al 78%**. Esto indica que el modelo logra un equilibrio entre precisión y recall, siendo efectivo incluso en presencia de cierto desbalance de clases.  
Además, la **precisión ponderada (≈ 81%)** confirma que cuando el modelo predice una clase, suele acertar, y la **recall ponderada (≈ 77%)** demuestra que identifica correctamente la mayoría de los casos positivos. Estas métricas respaldan que el modelo generaliza bien sobre los datos de prueba.

En el modelo no supervisado (KMeans), el **Silhouette score ≈ 0.72** sugiere una separación aceptable entre los clústeres, con estructuras bien definidas y poca mezcla entre grupos. Aunque no se cuenta con etiquetas reales para validar el agrupamiento, este valor indica que los patrones identificados por el modelo tienen coherencia interna.

En ambos casos, las métricas seleccionadas permiten evaluar de manera confiable el rendimiento sin procesar todos los datos manualmente, lo cual es fundamental al trabajar con grandes volúmenes de información usando Spark.
